# Moving Features Between Objects

- Inevitably, there will be times when you have distributed the functionality of something among many different classes sub-optimally
- These techniques focus on how to safely move functionality between classes, create new classes, and/or hide implementation details from public access

## 1. Move Method

- Problem
    - A method is used more in another class than in its own class.

- Solution
    - Move method to a class where it is used the most, then remove method from the original class, or at least make it a reference to the new location

- Motivation
    - Helps make classes more coherent when related data and methods are grouped together
    - If the moving of a method helps reduce outward calls to other classes, dependency is reduced

- How to refactor
    - Verify the features used by the old method, and see what else needs to be moved to the new class also. If a feature in the old class is used ONLY in the method to be moved, then move it along with the method to the new class.
        - Ensure that the method isn't declared in super/sub classes, or you'll have to handle it with polymorphism
    - Declare new method in recipient class
    - Decide how you want to refer to the new location of the method. Probably the ideal way is to have a new field to store an instance of the new class in the old class
    - If everything works, test deleting the old method
        - If it is too risky to delete, place a reference to the new method instead

- Relationships with other refactoring methods
    - Similar
        - Extract Method
        - Move Field
    - Related
        - Extract Class
        - Inline Class
        - Introduce Parameter Object

- Related code smells
    - Shotgun Surgery
    - Feature Envy
    - Switch Statements
    - Parallel Inheritance Hierachies
    - Message Chains
    - Inappropriate Intimacy
    - Data Class

- Example:

In [1]:
class bad__B():
    def do_something(self):
        print('123')

class bad__A():
    def __init__(self):
        self.B_Instance = bad__B()

    def do_something(self):
        self.B_Instance.do_something()


###

class good__A():
    def do_something(self):
        print('123')


## 2. Move Field

- Problem
    - A field is used more in another class than its own class

- Solution
    - Move it to the appropriate class

- Motivation
    - Minimise dependency, reduce objects calling others just to access specific fields

- How to refactor
    - If field is public, make it private and provide the access method.
    - Update the code to call the access method
    - Copy the field and access method into the new class
    - Replace all references to the old field
    - Delete the original field

- Relationships with other refactoring methods
    - Opposite 
        - Extract Class
        - Inline Class
    - Similar
        - Move Method

- Related code smells
    - Shotgun Surgery
    - Parallel Inheritance Hierachies 
    - Inappropriate Intimacy
    
- Example:

In [2]:
class bad__A():
    def __init__(self):
        self.a = 123
    
    ...

class bad__B():
    def __init__(self):
        self.bad__A = bad__A()

    def get_a_value(self):
        return self.bad__A.a
    
#####


class good__B():
    ## No dependency on another class, leaner object
    def __init__(self):
        self.a = 123

    def get_a_value(self):
        return self.a

## 3. Extract Class

- Problem
    - One class does the work of two, leading to awkward/cluttered class grouping

- Solution
    - Create a new class to handle the necessary fields/methods for a specific functionality

- Motivation
    - More logically separated code, single responsibility principle

- Drawbacks
    - Don't overdo it. If something is simple enough to leave as an inline code, just leave it. Or you'll end up with some classes that barely do anything

- How to refactor
    - Decide on how you want the functionality to be split
    - Decide on the relationship between the 2 classes. Try to keep it unidirectional
    - Move fields and methods to new class
    - Decide how you want your old class to access the method in the new class 

- Relationships with other refactoring methods
    - Opposite 
        - Inline Class
    - Similar
        - Extract Subclass
        - Replace data value with object

- Related code smells
    - Duplicate code
    - Large class
    - Divergent Change
    - Data Clumps
    - Primitive Obsession
    - Temporary Field
    - Inappropriate Intimacy

- Example:

In [13]:
class bad__Person():
    def __init__(self):
        self.name = 'abc'
        self.phone_country_code = '+65'
        self._phone_number = '91234567'
    
    @property
    def phone_number(self):
        return self.phone_country_code + self._phone_number
    
    @phone_number.setter
    def phone_number(self, value):
        if len(value) != 8:
            raise ValueError("invalid number")
        self._phone_number = value
    

p = bad__Person()
p.phone_number
# p.phone_number = '12345'
p.phone_number = '12345678'
p.phone_number

###

class good__HandPhone():
    def __init__(self):
        self.phone_country_code = '+65'
        self._phone_number = '91234567'

    @property
    def phone_number(self):
        return self.phone_country_code + self._phone_number
    
    @phone_number.setter
    def phone_number(self, value):
        if len(value) != 8:
            raise ValueError("invalid number")
        self._phone_number = value

class good__Person():
    def __init__(self):
        self.name = 'abc'
        self.phone = good__HandPhone()

    def print_phone_number(self):
        return self.phone.phone_number

p = good__Person()
p.print_phone_number()

'+6591234567'

## 4. Inline Class

- Problem
    - You have a class that does almost nothing, and no additional features are expected

- Solution
    - Remove the class, and move the logic to whereveer the class's current functionality is called

- Motivation
    - Minimise the number of classes floating around

- How to refactor
    - Move the unneeded class method into where it is called
    - Delete class and references to the class

- Relationships with other refactoring methods
    - Opposite 
        - Extract Class

- Related code smells
    - Shotgun Surgery
    - Lazy Class
    - Speculative Generality

- Example:

In [14]:
class bad__SquarePrinter():
    def __init__(self):
        ...

    def print_length(self, length):
        print(f'{length=}')

class bad__Square():
    def __init__(self, length):
        self.length = length
        self.square_printer = bad__SquarePrinter()
    
    def print_length(self):
        self.square_printer.print_length(self.length)

class good__Square():
    def __init__(self, length):
        self.length = length
        
    def print_length(self):
        print(f'{self.length=}')

## 5. Hide Delegate

- Problem
    - In a single method, you have a chained relationship between objects. So `method()` calls class `A` to recieve class `B`, then uses `B` to do something

- Solution
    - Instead of doing the chained call from `method()`, delegate it to class `A` so method does not rely on `B` directly

- Motivation
    - If you have a sequential call in this manner, the class containing `method()` will end up being dependent on many other classes. This is super not ideal, because if one part of your code changes, you always need to worry that this `method()`, though unrelated, may fail

- Drawback
    - Don't over delegate, or you'll end up with classes whose entire job is to call another class on your behalf

- How to refactor
    - If the logical chain goes from `method` -> `A` -> `B`, move the invocation of `B` to `A`, instead of doing it in method. This way, method only depends on `A`, and not on `B`

- Relationships with other refactoring methods
    - Opposite 
        - Remove middle man

- Related code smells
    - Message Chains
    - Inappropriate Intimacy

- Example:

In [ ]:
class bad__A():
    def __init__(self):
        self.B = bad__B()

class bad__B():
    def do_something(self):
        print('123')

class bad__client():
    def __init__(self):
        self.A = bad__A()

    def method(self):
        B = self.A.B
        B.do_something()
        
###

class good__A():
    def __init__(self):
        self.B = good__B()
    
    def do_something(self):
        self.B.do_something()

    ...

class good__B():
    def do_something(self):
        print('123')

class good__client():
    def __init__(self):
        self.A = good__A()

    def method(self):
        ## Dependency on B is entirely delegated to A. Any errors will come from A
        self.A.do_something()

## 6. Remove Middle Man

- Problem
    - A class's operations almost exclusively delegate methods to other classes

- Solution
    - Delegate these methods and force the client to call end methods directly

- Motivation
    - Don't clutter up code space with unnecessary classes

- How to refactor
    - Link the end-server class directly to the client classes, and remove references to the middle man class

- Relationships with other refactoring methods
    - Opposite 
        - Hide Delegate

- Related code smells
    - Middle Man
    
- Example:

In [ ]:
class B():
    def do_something(self):
        print('123')

class bad__A():
    def __init__(self):
        self.B = B()
    
    ## No other operation besides delegation
    def do_something(self):
        self.B.do_something()
    
class bad__client():
    def __init__(self):
        self.A = bad__A()

    def method(self):
        self.A.do_something()

### 

class good__client():
    def __init__(self):
        self.B = B()

    ## No reference to A needed. 1 less class 
    def method(self):
        self.B.do_something()

## 7. Introduce Foreign Method

- Problem
    - You are using some 3rd party code and you find that there is a functionality you'd like to introduce. However, for whatever reason, you are unable to modify the code, and the owners don't want to implement the feature

- Solution
    - Create a method in the client class that calls this 3rd party code that accepts an object from the utility code, and does whatever you want 
    - Basically you are extending the class's functionality by simply passing it as an object to a method that modifies it. Since you are passing the entire class, you have access to all class attributes and methods 

- Motivation
    - If the logic is used multiple times in your code, you will avoid repetitive code chunks

- Drawbacks
    - It won't always be obvious to people why you are doing something like this. If you find that you are using this pattern a lot, it's better to create a new class that wraps around the utility class you are using, and put the method there

- How to refactor
    - Create a new method in the client that takes in an object from the 3rd party class
    - Call the method to do whatever you want

- Relationships with other refactoring methods
    - Similar 
        - Introduce local extension

- Related code smells
    - Incomplete Library Class

- Example:

In [15]:
class ClosedDate():
    def __init__(self, date):
        self.previous_end = date

    def get_year(self):
        return self.previous_end[:4]

    def get_month(self):
        return self.previous_end[5:7]
    
    def get_day(self):
        return self.previous_end[8:10]
    
class Report():
    def __init__(self):
        self.previous_end = ClosedDate('2024-01-15')
    
    def bad__send_report(self):
        ## Awkward to get_day(), ideally we want to do self.previous_end.get_next_day()
        next_day = ClosedDate(f'{self.previous_end.get_year()}-{self.previous_end.get_month()}-{str(int(self.previous_end.get_day() + 1))}')
        ...

    def good__send_report(self, date: ClosedDate):
        ## Workaround, _get_next_day() method accepts a ClosedDate object
        next_day = self._get_next_day(self.previous_end)
        ...

    def _get_next_day(self, date: ClosedDate) -> ClosedDate:
        year = date.get_year()
        month = date.get_month()
        day = str(int(date.get_day() + 1))
        return ClosedDate(f'{year}-{month}-{day}')
    


## 8. Introduce Local Extension

- Problem
    - You are using some 3rd party code and you find that there is a functionality you'd like to introduce. However, for whatever reason, you are unable to modify the code, and the owners don't want to implement the feature.
    - In addition, this happens often, so you don't want to haphazardly just add methods that extend class operations like in `7. Introduce Foreign Method`

- Solution
    - Create a new class with the method extension that either inherits from or wraps around the 3rd party class

- Motivation
    - If the logic is used multiple times in your code, you will avoid repetitive code chunks
    - Grouping it in a class makes it clearer for maintainers why it was done

- How to refactor
    - Create a new class as a child or a wrapper of the utility class
    - Constructor should make use of the utility class 

- Relationships with other refactoring methods
    - Similar 
        - Introduce foreign method

- Related code smells
    - Incomplete Library Class

- Example:

In [ ]:
class ClosedDate():
    def __init__(self, date):
        self.previous_end = date

    def get_year(self):
        return self.previous_end[:4]

    def get_month(self):
        return self.previous_end[5:7]
    
    def get_day(self):
        return self.previous_end[8:10]

class MyClosedDate_Inherit(ClosedDate):
    def __init__(self, date):
        super().__init__(date)
    
    def get_next_day(self) -> 'MyClosedDate_Inherit':
        year = self.get_year()
        month = self.get_month()
        day = str(int(self.get_day() + 1))
        return MyClosedDate_Inherit(f'{year}-{month}-{day}')
    
class MyClosedDate_Wrap():
    def __init__(self, date):
        self.date = ClosedDate(date)
    
    def get_next_day(self) -> 'MyClosedDate_Wrap':
        year = self.date.get_year()
        month = self.date.get_month()
        day = str(int(self.date.get_day() + 1))
        return MyClosedDate_Wrap(f'{year}-{month}-{day}')

class Report():
    def __init__(self):
        self.previous_end_inherit = MyClosedDate_Inherit('2024-01-15')
        self.previous_end_wrap = MyClosedDate_Inherit('2024-01-15')
    
    def good__send_report(self):
        ## Awkward to get_day(), ideally we want to do self.previous_end.get_next_day()
        next_day_inherit = self.previous_end_inherit.get_next_day()
        next_day_wrap = self.previous_end_wrap.get_next_day()
        ...
    
